In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets evaluate bert_score nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/nlp_project/stage1_model"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/nlp_project/civitai_train_transformed.csv")
val_df = pd.read_csv("/content/drive/MyDrive/nlp_project/civitai_val_transformed.csv")
test_df = pd.read_csv("/content/drive/MyDrive/nlp_project/civitai_test_transformed.csv")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def format_caption_prompt(row):
    return f"### Caption:\n{row['gen_caption']}\n\n### Prompt:\n{row['prompt']}"

train_df["full_text"] = train_df.apply(format_caption_prompt, axis=1)
val_df["full_text"]   = val_df.apply(format_caption_prompt, axis=1)
test_df["full_text"]  = test_df.apply(format_caption_prompt, axis=1)

from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df[["full_text"]])
val_dataset   = Dataset.from_pandas(val_df[["full_text"]])
test_dataset  = Dataset.from_pandas(test_df[["full_text"]])

# Tokenize to only predict prompt
def tokenize(example):
    result = tokenizer(
        example["full_text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    result["labels"] = result["input_ids"].copy()
    return result

train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=["full_text"])
val_dataset   = val_dataset.map(tokenize, batched=True, remove_columns=["full_text"])
test_dataset  = test_dataset.map(tokenize, batched=True, remove_columns=["full_text"])

# Let it learn prompts
def mask_caption_tokens(example):
    input_ids = example["input_ids"]
    labels = example["labels"]
    caption_start = tokenizer("### Prompt:")["input_ids"][-1]

    try:
        start_idx = input_ids.index(caption_start)
    except ValueError:
        start_idx = 0

    labels = [-100 if i < start_idx else t for i, t in enumerate(labels)]
    return {"labels": labels}

train_dataset = train_dataset.map(mask_caption_tokens)
val_dataset   = val_dataset.map(mask_caption_tokens)
test_dataset  = test_dataset.map(mask_caption_tokens)

from torch.utils.data import DataLoader

def collate_fn(batch):
    return {
        "input_ids": torch.tensor([x["input_ids"] for x in batch]),
        "attention_mask": torch.tensor([x["attention_mask"] for x in batch]),
        "labels": torch.tensor([x["labels"] for x in batch])
    }

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

Map:   0%|          | 0/1058 [00:00<?, ? examples/s]

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Map:   0%|          | 0/1058 [00:00<?, ? examples/s]

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32022, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-06)
    (rotary_emb)

In [ ]:
from tqdm import tqdm

num_epochs = 3
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}")
    model.train()

    for step, batch in enumerate(tqdm(train_loader)):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        if input_ids.size(1) == 0:
            continue

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, use_cache=False)
        loss = outputs.loss

        if torch.isnan(loss):
            continue

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if step % 20 == 0:
            print(f"Step {step} | Loss: {loss.item():.4f}")


Epoch 1


  0%|          | 1/529 [00:01<14:50,  1.69s/it]

Step 0 | Loss: 5.9889


  4%|▍         | 21/529 [00:13<05:21,  1.58it/s]

Step 20 | Loss: 3.7775


...

 95%|█████████▍| 501/529 [04:52<00:17,  1.58it/s]

Step 500 | Loss: 0.7335


 98%|█████████▊| 521/529 [05:04<00:05,  1.58it/s]

Step 520 | Loss: 0.5813


100%|██████████| 529/529 [05:09<00:00,  1.71it/s]


In [ ]:
save_path = "/content/drive/MyDrive/nlp_project/stage2_prompt_enhancer"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/nlp_project/stage2_prompt_enhancer/tokenizer_config.json',
 '/content/drive/MyDrive/nlp_project/stage2_prompt_enhancer/special_tokens_map.json',
 '/content/drive/MyDrive/nlp_project/stage2_prompt_enhancer/tokenizer.json')

In [ ]:
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/nlp_project/stage2_prompt_enhancer"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Testing
input_caption = "futuristic city skyline"

inputs = tokenizer(input_caption, return_tensors="pt").to("cuda")
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=64)
generated_prompt = tokenizer.decode(output[0], skip_special_tokens=True)

print("Input:\n", input_caption)
print("\nGenerated Prompt:\n", generated_prompt)

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


Input:
 futuristic city skyline

Generated Prompt:
 futuristic city skyline at night, with towering skyscrapers and a large, glowing moon in a vibrant starry backdrop.

